### Creating and analyzing time-distance plots with stackplotX.py

In [22]:
# IMPORTS
%matplotlib notebook
import matplotlib
# matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import os, sys, re

import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta

import sunpy.map
from sunpy.net import Fido
from sunpy.net import attrs as a
import warnings
warnings.filterwarnings("ignore")
sys.path.insert(0, '/home/ivan/Study/Astro/solar')
import matplotlib.colors as colors


In [23]:
'''
To get stackplotX working properly in sunpy 5.0
replace the line
from sunpy.physics.solar_rotation import mapsequence_solar_derotate
with
from sunkit_image.coalignment import mapsequence_coalign_by_rotation
https://docs.sunpy.org/en/stable/whatsnew/4.0.html
'''
import stackplotX as stp

Loading AIA maps

In [17]:
obs_data_path = '/media/ivan/TOSHIBA EXT/aia_img/maps_part/'
start_time = Time('2011-03-07T13:45:26', scale='utc', format='isot')

In [18]:
mapsequence = sunpy.map.Map(obs_data_path + '/aia.lev1_euv_12s.2011-03-07T*.fits', sequence=True)

In [19]:
st = stp.Stackplot(mapsequence)  # Create a stackplotX Stackplot object

Use the mapseq_mkdiff method to create a difference map from your sequence.
'mode' refers to the type of difference you wish to calculate:
'rratio' for relative ratio,
'rdiff' for relative difference,
'dtrend' for detrend.
'dt' represents the time difference (in seconds) between the current frame and the reference frame.

In [20]:
st.mapseq_mkdiff(mode='rratio', dt=14.)

making the diff mapseq.....


100%|██████████| 78/78 [00:01<00:00, 46.31it/s]


ValueError: fp and xp are not of the same length.

MapSequence of 78 elements, with maps from AIAMap

Use the plot_mapseq method to visualize the difference map sequence.
Set 'diff=True' to indicate that a difference map is to be plotted.
'norm' refers to the color normalization option for the map.
Note: you can interact with the pop-up window to modify the map. A left-click adds a new anchor point for the slit on the map, while a right-click removes the latest modification.

In [21]:
st.plot_mapseq(diff=True, norm=colors.Normalize(vmin=0.1, vmax=2.4))

[-463.92158627 -113.46962336  202.38255521  551.13941367]


<AxesSubplot: title={'center': ' SDO AIA 131.0 Angstrom 2011-03-07T16:50:09.618'}, xlabel='X-position [arcsec]', ylabel='Y-position [arcsec]'>